In [1]:
import sys
import os
import gc

import pandas as pd
import numpy as np
import natsort
import random as rn
from tqdm import tqdm_notebook as tqdm
import tensorflow as tf
#import pyeeg
from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps

import matplotlib.pyplot as plt

#Keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Embedding, TimeDistributed, Dropout


#Sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPRegressor
from sklearn.svm import (SVC, SVR)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score
#from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
seed=42
np.random.seed(seed)
rn.seed(seed)
tf.set_random_seed(seed)
dir_path = os.getcwd()

# https://en.wikipedia.org/wiki/Neural_oscillation
SAMPLE_PER_SEC = 128
ALPHA_WAVE = [7.5, 12.5]
BETA_WAVE = [13, 30]
THETA_WAVE = [4, 8]
DELTA_WAVE = [1, 4]
LOW_GAMMA_WAVE = [30, 70]
HIGH_GAMMA_WAVE = [70, 150]

"""
Same for all
Min: -0.0033
Max: 0.0033


RAT 0
Class 1: 11240
Class 2: 8653
Class 3: 1707
Mean: -7.270247766565382e-06
Std: 0.00014033437213481842

RAT 1
Class 1: 11849
Class 2: 8647
Class 3: 1104
Mean: -4.883171278211801e-06
Std: 8.972252301644052e-05

RAT 2
Class 1: 11025
Class 2: 9833
Class 3: 742
Mean: -4.952989167390033e-06
Std: 0.00010911971573637134
"""



Using TensorFlow backend.


'\nSame for all\nMin: -0.0033\nMax: 0.0033\n\n\nRAT 0\nClass 1: 11240\nClass 2: 8653\nClass 3: 1707\nMean: -7.270247766565382e-06\nStd: 0.00014033437213481842\n\nRAT 1\nClass 1: 11849\nClass 2: 8647\nClass 3: 1104\nMean: -4.883171278211801e-06\nStd: 8.972252301644052e-05\n\nRAT 2\nClass 1: 11025\nClass 2: 9833\nClass 3: 742\nMean: -4.952989167390033e-06\nStd: 0.00010911971573637134\n'

In [ ]:
def get_train_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of 4 second sample. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path, csv_file)
    print(f"Reading {csv_file}")
    with open(csv_file, 'r') as csvfile:
        train_reader = pd.read_csv(csvfile)
        train_reader.drop(labels="Id", axis=1, inplace=True)
        
        
    return train_reader.values

def get_target_from_csv(csv_file):
    '''
    get a numpy array y of labels. the order follows the id of 4 second sample. 
    argument: relative path to the csv_file from the source folder.
    '''
    csv_file = os.path.join(dir_path,csv_file)
    with open(csv_file, 'r') as csvfile:
        label_reader = pd.read_csv(csvfile)
        #print("Labels: ", label_reader['id'])
        y = label_reader['y']
        
    y = np.array(y)
    return y

def get_features_emg_wavelet(X):
    """
     absolute EEG power in the 1–4Hz
    (delta), 5–9Hz (theta), 10–20Hz (low beta), and 30–40Hz (high beta) bands, absolute EMG
    power in the 1–10Hz band, theta-to-delta ratio, and beta-to-delta ratio 
    """
    all_featues = []
    for i in tqdm(range(X.shape[0])):
        features = list()
        x_i = X[i,:]
        delta = bandpower(x_i, DELTA_WAVE)
        features.append(delta)
        
        theta = bandpower(x_i, THETA_WAVE)
        features.append(theta)
        
        beta = bandpower(x_i, BETA_WAVE)
        features.append(beta)
        
        features.append(bandpower(x_i, ALPHA_WAVE))
        
        features.append(theta / delta)
        features.append(beta / delta)
    
        all_featues.append(features)
    return np.array(all_featues)

def get_features_emg(X):
    all_featues = []
    for i in tqdm(range(X.shape[0])):
        features = list()
        # https://ieeexplore.ieee.org/document/7748960
        x_i = X[i,:]
        # Root Mean Square (RMS): RMS of EMG
        features.append(mean_squared_error(x_i, np.zeros(x_i.shape)))
        
        #Integrated Absolute Value (IAV)
        features.append(np.sum(np.abs(x_i)))
        
        # Mean Absolute Value (MAV): MAV feature can be expressed as
        features.append(np.mean(x_i))
        
        # TBD:
        # Modified Mean Absolute Value type 1
        # Modified Mean Absolute Value type 2
        
        # Simple Square Integral (SSI): SSI is calculated as
        features.append(np.sum(x_i ** 2))
        
        # Variance (VAR): VAR is calculated as
        features.append(np.var(x_i))
        
        #The 3rd, 4th and 5th temporal moments
        features.append(np.mean(x_i ** 3))
        features.append(np.mean(x_i ** 4))
        features.append(np.mean(x_i ** 5))
        
        # TBD
        # v-Order 
        
        # Waveform Length
        features.append(np.sum(np.abs(np.diff(x_i))))
        
        # Average Amplitude Change
        features.append(np.mean(np.abs(np.diff(x_i))))
        
        # Difference Absolute Standard Deviation Value
        features.append(np.sqrt(np.mean(np.power(np.diff(x_i), 2))))
        
        # AX BASIC FEATUERS
        features.append(np.std(x_i))
        features.append(np.min(x_i))
        features.append(np.max(x_i))
        features.append(np.sum(x_i < 0.0005))
        
        all_featues.append(features)
    return np.array(all_featues)

def get_features_eeg(X):
    all_featues = []
    # NOT SURE ABOUT THIS VALUES 
    # LETS DOUBLE CHECK
    K_MAX = 6
    FREQ_BANDS = list(range(16))
    TAU = 16
    # embedding dimension
    DE = 32
    
    for i in tqdm(range(X.shape[0])):
        features = list()
        # http://pyeeg.sourceforge.net/
        x_i = X[i,:]
        
        ## !IMPORTANT! => 
        # Power Spectral Intensity (PSI) and Relative Intensity Ratio (RIR)	bin_power()	Two 1-D vectors
        
        
        # Petrosian Fractal Dimension (PFD)	pdf()	A scalar
        features.append(pyeeg.pfd(x_i))
        
        # Higuchi Fractal Dimension (HFD)	hfd()	A scalar
        features.append(pyeeg.hfd(x_i, K_MAX))
        
        # Hjorth mobility and complexity	hjorth()	Two scalars
        
        # Spectral Entropy (Shannon's entropy of RIRs)	spectral_entropy()	A scalar
        #features.append(pyeeg.spectral_entropy(x_i, FREQ_BANDS, SAMPLE_PER_SEC))
        
        # SVD Entropy	svd_entropy()	A scalar
        #features.append(pyeeg.svd_entropy(x_i, TAU, DE))
        
        # Fisher Information	fisher_info()	A scalar
        features.append(pyeeg.fisher_info(x_i, TAU, DE))
          
        # Detrended Fluctuation Analysis (DFA)	dfa()	A scalar
        features.append(pyeeg.dfa(x_i))
        
        # Hurst Exponent (Hurst)	hurst()	A scalar
        #features.append(pyeeg.hurst(x_i))
        
        # AX BASIC FEATUERS
        features.append(np.mean(x_i))
        features.append(np.std(x_i))
        features.append(np.min(x_i))
        features.append(np.max(x_i))
        features.append(np.sum(np.abs(x_i) < 0.00005))
        
        
        
        all_featues.append(features)
    return np.array(all_featues)

def split_train_validation(X, y, valid=2):
    """
    3 test subjects in train => split into 2|1
    """
    sample_count_per_subject = int(X.shape[0] / 3)
    X_train = X[:sample_count_per_subject * valid]
    y_train = y[:sample_count_per_subject * valid]
    X_valid = X[sample_count_per_subject * valid:]
    y_valid = y[sample_count_per_subject * valid:]
    
    return X_train, y_train, X_valid, y_valid


def bandpower(data, band, window_sec=4, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.

    Examples
    ------
    1. Absolute and relative power in the delta band
        >>> delta = bandpower(data, 100, [0.5, 4])
        >>> delta_relative = bandpower(data, 100, [0.5, 4], relative=True)

    2. Delta / beta ratio
        >>> window_sec = 4
        >>> delta = bandpower(data, 100, [0.5, 4], window_sec)
        >>> beta = bandpower(data, 100, [12, 30], window_sec)
        >>> db_ratio = delta / beta
    """
    
    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if window_sec is not None:
        nperseg = window_sec * SAMPLE_PER_SEC
    else:
        nperseg = (2 / low) * SAMPLE_PER_SEC

    freqs, psd = welch(data, SAMPLE_PER_SEC, nperseg=nperseg, scaling='density')

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs < high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], freqs[idx_band])

    if relative:
        bp /= simps(psd, freqs)
    return bp

def plot_PCA_clusters(X, y_train):
    pca = PCA(n_components=2)
    pca.fit(X)
    existing_2d = pca.transform(X)
    awake_fold = existing_2d[y_train == 1]
    sleep_fold_nrem = existing_2d[y_train == 2]
    sleep_fold_rem = existing_2d[y_train == 3]

    f, (ax1, ax2, ax3) = plt.subplots(1, 3)
    ax1.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    ax2.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    ax3.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()

    plt.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    plt.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    plt.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()
    
def plot_fct_spec(data):
    time = np.arange(len(data)) / SAMPLE_PER_SEC
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    plt.plot(time, data, lw=1.5, color='k')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Voltage')
    plt.xlim([time.min(), time.max()])
    plt.title('N3 sleep EEG data (F3)')

    win = 4 * SAMPLE_PER_SEC
    freqs, psd = signal.welch(data, SAMPLE_PER_SEC, nperseg=win, scaling='density')

    # Plot the power spectrum

    plt.figure(figsize=(8, 4))
    plt.plot(freqs, psd, color='k', lw=2)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power spectral density (V^2 / Hz)')
    plt.ylim([0, psd.max() * 1.1])
    plt.title("Welch's periodogram")
    plt.xlim([0, 20])
    
def plot_TSNE_clusters(X, y_train):
    N_SAMPLES = 2500
    random_index = np.random.choice(X.shape[0], N_SAMPLES)
    X = X[random_index, :]
    y_train = y[random_index]

    existing_2d = TSNE(n_components=2).fit_transform(X)
    awake_fold = existing_2d[y_train == 1]
    sleep_fold_nrem = existing_2d[y_train == 2]
    sleep_fold_rem = existing_2d[y_train == 3]

    f, (ax1, ax2, ax3) = plt.subplots(1, 3)
    ax1.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    ax2.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    ax3.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()

    plt.plot(awake_fold[:, 0], awake_fold[:, 1], 'r.')
    plt.plot(sleep_fold_nrem[:, 0], sleep_fold_nrem[:, 1], 'b.')
    plt.plot(sleep_fold_rem[:, 0], sleep_fold_rem[:, 1], 'y.')
    plt.show()
    
def plot_signal(data):
    time = np.arange(len(data)) / SAMPLE_PER_SEC
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    plt.plot(time, data, lw=1.5, color='k')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Voltage')
    plt.xlim([time.min(), time.max()])
    plt.title('N3 sleep EEG data (F3)')
    
def get_data_of_rat(X, y, i):
    sample_cnt = int(X.shape[0] / 3)
    if i == 0:
        return X[:sample_cnt, :], y[:sample_cnt]
    if i == 1:
        return X[sample_cnt: 2 * sample_cnt, :], y[sample_cnt: 2 * sample_cnt]
    if i == 2:
        return X[2*sample_cnt:, :], y[2*sample_cnt:]
    

In [7]:
# Load the data
# train
train_emg = os.path.join("/content/drive/data/data/train", "train_emg.csv")
train_eeg_1 = os.path.join("/content/drive/data/data/train", "train_eeg1.csv")
train_eeg_2 = os.path.join("/content/drive/data/data/train", "train_eeg2.csv")

# trest
test_emg = os.path.join("/content/drive/data/data/test", "test_emg.csv")
test_eeg_1 = os.path.join("/content/drive/data/data/test", "test_eeg1.csv")
test_eeg_2 = os.path.join("/content/drive/data/data/test", "test_eeg2.csv")

# labels
train_target = os.path.join("/content/drive/data/data/train", "train_labels.csv")

x_train_emg = get_train_from_csv(train_emg) #List of numpy arrays
x_train_eeg_1 = get_train_from_csv(train_eeg_1) #List of numpy arrays
x_train_eeg_2 = get_train_from_csv(train_eeg_2) #List of numpy arrays
y_train = get_target_from_csv(train_target) #Numpy array of labels

x_test_emg = get_train_from_csv(test_emg) #List of numpy arrays
x_test_eeg_1 = get_train_from_csv(test_eeg_1) #List of numpy arrays
x_test_eeg_2 = get_train_from_csv(test_eeg_2) #List of numpy arrays

print(f"Class 1: {np.sum(y_train == 1)}")
print(f"Class 2: {np.sum(y_train == 2)}")
print(f"Class 3: {np.sum(y_train == 3)}")


Reading /content/drive/data/data/train/train_emg.csv
Reading /content/drive/data/data/train/train_eeg1.csv
Reading /content/drive/data/data/train/train_eeg2.csv
Reading /content/drive/data/data/test/test_emg.csv
Reading /content/drive/data/data/test/test_eeg1.csv
Reading /content/drive/data/data/test/test_eeg2.csv
Class 1: 34114
Class 2: 27133
Class 3: 3553


In [ ]:
def check_for_floats(Array):
    for row in Array:
        for el in row:
            if isinstance(el, float):
                print(el)
                
def from_label_to_vec(labels):
    labels_vec = []
    for l in labels:
        if l == 1:
            labels_vec.append([1,0,0])
        elif l == 2:
            labels_vec.append([0,1,0])
        elif l == 3:
            labels_vec.append([0,0,1])
    return labels_vec

                
def from_vec_to_labels(vecs):
    labels = []
    for v in vecs:
        if v[0] == 1:
            labels.append(1)
        elif v[1] == 1:
            labels.append(2)
        elif v[2] == 1:
            labels.append(3)
    return labels
  
def count_labels(labels):
    classes = [0,0,0]
    for l in labels:
      classes[l-1]+=1
    print(classes)
            

In [11]:
gc.collect() 

print(x_train_eeg_1.shape)
print(x_train_eeg_2.shape)
print(x_train_emg.shape)
print(y_train.shape)

X_total_train = np.concatenate((x_train_eeg_1,x_train_eeg_2, x_train_emg), axis=1)*1000000
X_total_test = np.concatenate((x_test_eeg_1,x_test_eeg_2, x_test_emg), axis=1)*1000000

print(X_total_train.shape)
print(X_total_test.shape)


sc = StandardScaler()
sc.fit(np.concatenate((X_total_train,X_total_test)))

X_total_train_scaled = sc.transform(X_total_train)
X_total_test_scaled = sc.transform(X_total_test)

(64800, 512)
(64800, 512)
(64800, 512)
(64800,)
(64800, 1536)
(43200, 1536)


In [12]:
gc.collect()

#Labels needed as probability vectors for the softmax
Y = from_label_to_vec(y_train)

#Just selecting the train / validation set

X_train = np.concatenate((X_total_train_scaled[0:21600],X_total_train_scaled[21600*2:]),axis=0)
Y_train = np.concatenate((Y[0:21600],Y[21600*2:]),axis=0)

X_valid = X_total_train_scaled[21600:21600*2]
Y_valid = Y[21600:21600*2]

print(X_train.shape)
print(X_valid.shape)

Y_train = np.array(Y_train)
Y_valid = np.array(from_vec_to_labels(Y_valid))


gc.collect()

(43200, 1536)
(21600, 1536)
(43200, 1, 1536)
[[0 1 0]
 [0 1 0]
 [0 1 0]
 ...
 [1 0 0]
 [1 0 0]
 [1 0 0]]
[3 3 3 ... 1 1 1]


In [13]:
X_train = np.reshape(X_train, (43200,1536))
X_valid = np.reshape(X_valid, (21600,1536))

model = Sequential()
model.add(Dense(1000, input_dim = num_steps))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))
model.add(Dense(1000))
model.add(keras.layers.LeakyReLU(alpha=0.1))
model.add(Dropout(0.25))
model.add(Dense(3))
model.add(Activation('softmax'))
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              1537000   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 1000)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 3003      
__________

In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(x=X_train, y=Y_train, epochs=3, verbose=1, shuffle=False)
#model.fit(x=X_total_train_scaled, y=np.array(Y), epochs=3, verbose=1, validation_split=0.1, shuffle=True)

Epoch 1/3
43200/43200 [==============================] - 28s 646us/step - loss: 0.4161 - categorical_accuracy: 0.8326
Epoch 2/3
43200/43200 [==============================] - 26s 609us/step - loss: 0.1686 - categorical_accuracy: 0.9470
Epoch 3/3
43200/43200 [==============================] - 26s 609us/step - loss: 0.0793 - categorical_accuracy: 0.9785


In [30]:
y_pred = model.predict(X_valid)
labels = []
for p in preds:
    labels.append(np.argmax(p)+1)
labels = np.array(labels)
print(labels)

[3 2 3 ... 2 1 1]


In [31]:
BMAC = accuracy_score(Y_valid, labels)
print(BMAC)
print(Y_valid)
print(labels)
count_labels(labels)

0.8063888888888889
[3 3 3 ... 1 1 1]
[3 2 3 ... 2 1 1]
[8443, 12020, 1137]


In [27]:
X_train = np.reshape(X_train, (43200,1,1536))
X_valid = np.reshape(X_valid, (21600,1,1536))

#LSTM parameters
hidden_size = 500
use_dropout = True
num_steps = len(X_total_train_scaled[0])



model = Sequential()
#model.add(Embedding(input_length=num_steps, input_dim = (num_steps,), output_dim = hidden_size))
model.add(LSTM(hidden_size, return_sequences=False, input_dim = num_steps))
#model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))
print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(500, return_sequences=False, input_shape=(None, 153...)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 500)               4074000   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 1503      
_________________________________________________________________
activation_3 (Activation)    (None, 3)                 0         
Total params: 4,075,503
Trainable params: 4,075,503
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model.fit(x=X_train, y=Y_train, batch_size=1, epochs=1, verbose=1, shuffle=False)

Epoch 1/1
43200/43200 [==============================] - 674s 16ms/step - loss: 0.5153 - categorical_accuracy: 0.7934


In [16]:
y_pred = model.predict(X_valid)
labels = []
for p in preds:
    labels.append(np.argmax(p)+1)
labels = np.array(labels)
print(labels)


[3 2 3 ... 2 1 1]


In [18]:
BMAC = accuracy_score(Y_valid, labels)
print(BMAC)
print(Y_valid)
print(labels)
count_labels(labels)

    100% |████████████████████████████████| 849kB 21.5MB/s 
  Running setup.py bdist_wheel for metrics ... done
  Stored in directory: /root/.cache/pip/wheels/38/e8/55/c4c648451a33a60920318eba4c5735088dbf78dc20d4277db0
  Running setup.py bdist_wheel for pathspec ... done
  Stored in directory: /root/.cache/pip/wheels/d1/2e/6b/c67842adf3b13ea33ca3fe6a2a963622821ad647db724688e2
Successfully built metrics pathspec
jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.15 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: Pygments 2.1.3
    Uninstalling Pygments-2.1.3:
      Successfully uninstalled Pygments-2.1.3
0.8063888888888889
[3 3 3 ... 1 1 1]
[3 2 3 ... 2 1 1]
[8443, 12020, 1137]


In [ ]:
submission_name = "fv_keras.csv"

print(f"Class 1: {np.sum(y_pred == 1)}")
print(f"Class 2: {np.sum(y_pred == 2)}")
print(f"Class 3: {np.sum(y_pred == 3)}")

y_pred_df = pd.DataFrame(y_pred)
y_pred_df = y_pred_df.assign(Id=list(range(y_pred.shape[0])))
y_pred_df.columns = ['y', 'Id']
display(y_pred_df)


submission_folder = os.path.join(dir_path,"submissions/")
csv_file = submission_folder + submission_name

with open(csv_file, 'w') as csv:
    y_pred_df.to_csv(csv,index = False)
"""
Class 1: 23933
Class 2: 18553
Class 3: 714
"""